In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path+"\\Source")

from Source.Agent import Agent, DeltaHedger
from Source.Market import Market, StockGeometricBrownianMotion, EuropeanCallOption

We create a Martingale GBM. We verify the process is a martingale by Monte Carlo Simulation. The initial stock price is 100, and we expect the average of stocks price  is  100 at T=100. We simulate 10000 times and calculate the average.

In [2]:
final_stock_price_list = []
for _ in range(10000):
    stock = StockGeometricBrownianMotion('stock_gbm', 100, -0.02/252, 0.2 / np.sqrt(252))  # Martingale GBM
    for time in range(1, 100):
        stock.evolve(time)
    final_stock_price_list.append(stock.current_value)
print(f'Mean of stock final values: {np.mean(final_stock_price_list):.3f}')
print(f'Std of stock final values: {np.std(final_stock_price_list):.3f}')

Mean of stock final values: 99.740
Std of stock final values: 12.555


Suppose the stock price follows a Martingale GBM process with annual volatility 20%. An option trader holds 10 ATM options with two weeks expiry. The trader decides to delta hedge her option positions everyday. We will use this example to illustrate her delta hedging process.

In [4]:
stock = StockGeometricBrownianMotion('stock_gbm', 100, -0.02 / 252, 0.2 / np.sqrt(252))
option = EuropeanCallOption('option', [stock], 100, 10)
market = Market([stock, option])

delta_hedging_initial_asset = {'Cash': 10000, 'stock_gbm': 0, 'option': 10}  # agent holds an option
delta_hedging_trader = DeltaHedger('Agent', delta_hedging_initial_asset)

# set up initial hedging
print('\nTime=', 0)
market.mark_current_value_to_record(0)
delta_hedging_trader.generate_delta_hedging_plans(market)
delta_hedging_trader.trade(market, 0, print_log=True)
delta_hedging_trader.evaluate_holding_asset_values(market, print_log=True)
delta_hedging_trader.generate_performance_report(market, 0)

for time in range(1, 11):
    print('\nTime=', time)
    market.evolve(time)
    market.mark_current_value_to_record(time)
    delta_hedging_trader.evaluate_holding_asset_values(market, print_log=True)
    delta_hedging_trader.generate_performance_report(market, time)
    delta_hedging_trader.generate_delta_hedging_plans(market)
    delta_hedging_trader.trade(market, time, print_log=True)

print('\nEnd of Hedging')
delta_hedging_trader.evaluate_holding_asset_values(market, print_log=True)
print(f'\nCumulative PnL: ${delta_hedging_trader.historical_performance[10].cumulative_pnl:.3f}')


Time= 0
Agent: sell 5.0 stock_gbm, Cash + $500.000
Agent, Holding:
Cash: $10500.0
stock_gbm: -5.0 * $100.000 = $-500.000
option: 10 * $1.589 = $15.893
Total: $10015.893

Time= 1
Agent, Holding:
Cash: $10500.0
stock_gbm: -5.0 * $101.139 = $-505.697
option: 10 * $2.154 = $21.536
Total: $10015.840
Agent: sell 1.0 stock_gbm, Cash + $101.139

Time= 2
Agent, Holding:
Cash: $10601.139355794781
stock_gbm: -6.0 * $100.707 = $-604.239
option: 10 * $1.808 = $18.076
Total: $10014.976

Time= 3
Agent, Holding:
Cash: $10601.139355794781
stock_gbm: -6.0 * $103.054 = $-618.325
option: 10 * $3.392 = $33.924
Total: $10016.738
Agent: sell 2.0 stock_gbm, Cash + $206.108

Time= 4
Agent, Holding:
Cash: $10807.247609155635
stock_gbm: -8.0 * $105.860 = $-846.878
option: 10 * $5.900 = $59.002
Total: $10019.371
Agent: sell 2.0 stock_gbm, Cash + $211.720

Time= 5
Agent, Holding:
Cash: $11018.967121592948
stock_gbm: -10.0 * $108.393 = $-1083.928
option: 10 * $8.395 = $83.946
Total: $10018.985

Time= 6
Agent, Hold

We compare the pnl differences between 3 delta hedging strategy:
1. Daily delta hedging
2. Only initial hedging
3. Non delta hedging

In [6]:
def simulate_delta_hedge_pnl(num_trails):
    final_pnl_list = []
    for _ in range(num_trails):
        stock = StockGeometricBrownianMotion('stock_gbm', 100, -0.02 / 252, 0.2 / np.sqrt(252))
        option = EuropeanCallOption('option', [stock], 100, 10)
        market = Market([stock, option])
        
        delta_hedging_initial_asset = {'Cash': 10000, 'stock_gbm': 0, 'option': 10}  # agent holds an option
        delta_hedging_trader = DeltaHedger('Agent', delta_hedging_initial_asset)
        
        # set up initial hedging
        market.mark_current_value_to_record(0)
        delta_hedging_trader.generate_delta_hedging_plans(market)
        delta_hedging_trader.trade(market, 0)
        delta_hedging_trader.evaluate_holding_asset_values(market)
        delta_hedging_trader.generate_performance_report(market, 0)
        
        for time in range(1, 11):
            market.evolve(time)
            market.mark_current_value_to_record(time)
            delta_hedging_trader.evaluate_holding_asset_values(market)
            delta_hedging_trader.generate_performance_report(market, time)
            delta_hedging_trader.generate_delta_hedging_plans(market)
            delta_hedging_trader.trade(market, time)
        
        final_pnl_list.append(delta_hedging_trader.historical_performance[10].cumulative_pnl)
    return final_pnl_list


def simulate_only_initial_delta_hedge_pnl(num_trails):
    final_pnl_list = []
    for _ in range(num_trails):
        stock = StockGeometricBrownianMotion('stock_gbm', 100, -0.02 / 252, 0.2 / np.sqrt(252))
        option = EuropeanCallOption('option', [stock], 100, 10)
        market = Market([stock, option])
        
        delta_hedging_initial_asset = {'Cash': 10000, 'stock_gbm': 0, 'option': 10}  # agent holds an option
        delta_hedging_trader = DeltaHedger('Agent', delta_hedging_initial_asset)
        
        # set up initial hedging
        market.mark_current_value_to_record(0)
        delta_hedging_trader.generate_delta_hedging_plans(market)
        delta_hedging_trader.trade(market, 0)
        delta_hedging_trader.evaluate_holding_asset_values(market)
        delta_hedging_trader.generate_performance_report(market, 0)
        
        for time in range(1, 11):
            market.evolve(time)
            market.mark_current_value_to_record(time)
            delta_hedging_trader.evaluate_holding_asset_values(market)
            delta_hedging_trader.generate_performance_report(market, time)
        
        final_pnl_list.append(delta_hedging_trader.historical_performance[10].cumulative_pnl)
    return final_pnl_list


def simulate_non_hedging_pnl(num_trails):
    final_pnl_list = []
    for _ in range(num_trails):
        stock = StockGeometricBrownianMotion('stock_gbm', 100, -0.02 / 252, 0.2 / np.sqrt(252))
        option = EuropeanCallOption('option', [stock], 100, 10)
        market = Market([stock, option])
        
        delta_hedging_initial_asset = {'Cash': 10000, 'stock_gbm': 0, 'option': 10}  # agent holds an option
        delta_hedging_trader = DeltaHedger('Agent', delta_hedging_initial_asset)
        
        market.mark_current_value_to_record(0)
        delta_hedging_trader.evaluate_holding_asset_values(market)
        delta_hedging_trader.generate_performance_report(market, 0)
        
        for time in range(1, 11):
            market.evolve(time)
            market.mark_current_value_to_record(time)
            delta_hedging_trader.evaluate_holding_asset_values(market)
            delta_hedging_trader.generate_performance_report(market, time)
        
        final_pnl_list.append(delta_hedging_trader.historical_performance[10].cumulative_pnl)
    return final_pnl_list


delta_hedge_pnl = simulate_delta_hedge_pnl(num_trails=10000)
print('Daily Delta Hedger:')
print(f'Average Cumulative PnL: ${np.mean(delta_hedge_pnl):.3f}')
print(f'Std Cumulative PnL: ${np.std(delta_hedge_pnl):.3f}')

delta_hedge_pnl = simulate_delta_hedge_pnl(num_trails=10000)
print('\nTrader who only does Initial Hedge:')
print(f'Average Cumulative PnL: ${np.mean(delta_hedge_pnl):.3f}')
print(f'Std Cumulative PnL: ${np.std(delta_hedge_pnl):.3f}')

non_hedge_pnl = simulate_non_hedging_pnl(num_trails=10000)
print('\nTrader who Never Hedge:')
print(f'Average Cumulative PnL: ${np.mean(non_hedge_pnl):.3f}')
print(f'Std Cumulative PnL: ${np.std(non_hedge_pnl):.3f}')

Daily Delta Hedger:
Average Cumulative PnL: $-0.080
Std Cumulative PnL: $4.343



Trader who only does Initial Hedge:
Average Cumulative PnL: $-0.055
Std Cumulative PnL: $4.429



Trader who Never Hedge:
Average Cumulative PnL: $-0.131
Std Cumulative PnL: $23.465


According to simulation results, initial delta hedging can reduce the most of delta risk from an ATM option. When time to maturity goes smaller, ATM gamma gets greater. The delta hedging trader needs to hedge more frequently, in other words, hedging in intraday frequency. As a result, daily delta hedge can only reduce risk slightly from initial hedging.